In [ ]:
# ptorch + Src Image to validate
import torch
import torchvision.transforms as transforms
import os
from PIL import Image

model_file = 'resnet50'
model = torch.load(model_file + '.pth')
device = torch.device('cpu')

model.to(device)
model.eval()

# 定义图像预处理步骤
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 定义推理函数
def infer_image(image_path):
    image = Image.open(image_path)
    image = image.convert('RGB')
    image = preprocess(image)
    image = image.unsqueeze(0)  # 添加批次维度
    with torch.no_grad():
        output = model(image)
    _, predicted = torch.max(output, 1)
    return predicted.item()

# 加载 labels.txt 文件并解析标签
def load_labels(labels_file):
    labels = {}
    with open(labels_file, 'r') as f:
        for line in f:
            image_name, label = line.strip().split(',')
            labels[image_name] = int(label)
    return labels

# 计算准确率
def calculate_accuracy(validation_dir, labels_file):
    labels = load_labels(labels_file)
    correct = 0
    total = 0
    
    for image_name in os.listdir(validation_dir):
        if image_name.lower().endswith('.jpg'):
            image_path = os.path.join(validation_dir, image_name)
            if os.path.isfile(image_path):
                predicted_label = infer_image(image_path)
                true_label = labels.get(image_name)
                if true_label is not None:
                    if predicted_label == true_label:
                        correct += 1
                    total += 1
    
    accuracy = correct / total if total > 0 else 0
    return accuracy

validation_dir = r'data\imagenet\validation'
calculate_accuracy(validation_dir, os.path.join(validation_dir, '_labels.txt'))


C:\Users\fangyangci\AppData\Local\Temp\ipykernel_6184\3662643089.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_file + '.pth')


0.7666666666666667

In [2]:
# ptorch + data.npz to validate
import torch
import numpy as np
import os

model_file = 'resnet50'
model = torch.load(model_file + '.pth')
device = torch.device('cpu')

model.to(device)
model.eval()

validation_dir = r'data\imagenet\validation'

# 加载npz文件中的数据
data = np.load(os.path.join(validation_dir, '_data.npz'))
images = data['images']
labels = data['labels']

with torch.no_grad():
    output = model(torch.from_numpy(images))
_, predicted = torch.max(output, 1)

accuracy = (predicted.numpy() == labels).mean()
print(accuracy)

C:\Users\fangyangci\AppData\Local\Temp\ipykernel_6184\1408581544.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_file + '.pth')


0.77
